# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования:**  Разобраться влияет ли количество детей клиента и семейное положение на факт погашения кредита в срок.

**Ход исследования**
1. Обзор данных
2. Предобработка данных
3. Ответы на вопросы

## Шаг 1. Откройте файл с данными и изучите общую информацию 

In [1]:
import pandas as pd
try:
    data = pd.read_csv('data.csv') #чтение файла и запись в переменную data
except:
    data = pd.read_csv('/datasets/data.csv')

In [2]:
data.head(10) #просмотр первых 10 строк

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


В таблице 12 столбцов:

**children** - количество детей у клиента

**days_employed** - общий трудовой стаж в днях

**dob_years** - возраст клиента

**education** - уровень образования клиента

**education_id** - идентификатор уровеня образования клиента

**family_status** - семейное положение

**family_status_id** - идентификатор семейного положения

**gender** - пол клиента

**income_type** - тип занятости

**debt** - имел ли клиент задолженность по возврату кредитов

**total_income** - ежемесясчный доход

**purpose** - цель получения кредита

In [3]:
data.info() #получение общей информации о данных в data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
data.isna().sum() #информация о количестве пропусков в каждом столбце

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
data.isna().mean() #долевое соотношение пропусков относительно общей длины датафрейма

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

In [6]:
data.describe() #статистика по датасету

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [7]:
columns_name = ['children', 'family_status', 'purpose', 'gender']

for column in columns_name: #цикл, который выводит на экарн уникальные значения столбцов в переменной columns_name
    print(column, data[column].unique()) 
    print()

children [ 1  0  3  2 -1  4 20  5]

family_status ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']

purpose ['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образовани

In [8]:
data[data['children'] == -1] # таблица с клиентами, у которых значение в столбце children равно -1

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
291,-1,-4417.703588,46,среднее,1,гражданский брак,1,F,сотрудник,0,102816.346412,профильное образование
705,-1,-902.084528,50,среднее,1,женат / замужем,0,F,госслужащий,0,137882.899271,приобретение автомобиля
742,-1,-3174.456205,57,среднее,1,женат / замужем,0,F,сотрудник,0,64268.044444,дополнительное образование
800,-1,349987.852217,54,среднее,1,Не женат / не замужем,4,F,пенсионер,0,86293.724153,дополнительное образование
941,-1,NaN,57,Среднее,1,женат / замужем,0,F,пенсионер,0,NaN,на покупку своего автомобиля
1363,-1,-1195.264956,55,СРЕДНЕЕ,1,женат / замужем,0,F,компаньон,0,69550.699692,профильное образование
1929,-1,-1461.303336,38,среднее,1,Не женат / не замужем,4,M,сотрудник,0,109121.569013,покупка жилья
2073,-1,-2539.761232,42,среднее,1,в разводе,3,F,компаньон,0,162638.609373,покупка жилья
3814,-1,-3045.290443,26,Среднее,1,гражданский брак,1,F,госслужащий,0,131892.785435,на проведение свадьбы
4201,-1,-901.101738,41,среднее,1,женат / замужем,0,F,госслужащий,0,226375.766751,операции со своей недвижимостью


In [9]:
data['gender'].unique() #уникальные значения в столбце gender

array(['F', 'M', 'XNA'], dtype=object)

In [10]:
data[data['gender'] == 'XNA'] # клиенты, у которых значение в столбце gender равно 'XNA'

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


**Вывод:**  
При изучении общей информации о датафрейме были выявлены следующие проблемы в данных:

- в столбцах days_employed и children некоторые данные записаны с минусом. Так же столбец days_employed лучше привести к целочисленному типу данных, так как в данном столбце речь идет об отработанных клиентом днях.

- в столбце education одна и та же информация записана по-разному (с разным регистром). Необходимо будет привести все данные столбца к нижнему регистру.

- в столбцах days_employed и total_income есть пропуски и их одинаковое количество. Их нужно будет заменить на числовое значение.
 
- в столбце purpose, где описываются цели получения кредита, слишком много похожих пунктов. Необходимо будет применить лемматизацию. 

- у одного клиента значение в столбце gender равно XNA, возможно кто-то из сотрудников банка просто ошибся при указании гендера. Заменить это на значение F или M не получится, так как по другим данным не возможно определить пол клиента. Но это не как не повлияет на исследование, и можно оставить это значение как есть.

## Шаг 2. Предобработка данных   <a id ="preprocessing"></a>

### Обработка пропусков

In [11]:
data['days_employed'] = abs(data['days_employed']) #заменяем отрицательные числа на положительные в столбце days_employed
data['children'] = abs(data['children']) #заменяем отрицательные числам на положительные в столбце children

data.isna().sum() # посчитаем количество пропусков в датафрейме

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [12]:
nan = data[data['days_employed'].isna() == True] # в переменую nan запишем все строчки, в которых есть пропуски

nan['income_type'].unique() #уникальные значения столбца income_type в датафреме с только пропущенными значениями

array(['пенсионер', 'госслужащий', 'компаньон', 'сотрудник',
       'предприниматель'], dtype=object)

In [13]:
data['income_type'].unique() #уникальные значения столбца income_type в исходном датафрейме

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [14]:
data.groupby('income_type')['children'].count() #количество людей разных типов занятости

income_type
безработный            2
в декрете              1
госслужащий         1459
компаньон           5085
пенсионер           3856
предприниматель        2
сотрудник          11119
студент                1
Name: children, dtype: int64

Здесь можно увидеть, что людей с типом занятости "безработный", "в декрете", "предприниматель", "студент" в исходном датафрейме не больше двух человек.

In [15]:
median_days = data.groupby('income_type')['days_employed'].median() #ищем медианное значения количества рабочих дней
#для всех групп столбца income_type

median_income = data.groupby('income_type')['total_income'].median() #ищем медианное значения зарплаты
#для всех групп столбца income_type

In [16]:
median_income = data.groupby('income_type')['total_income'].median() #ищем медианное значения зарплаты
#для всех групп столбца income_type

In [17]:
def filling (df, median_days_employed, median_total_income): #функция для замены пропусков на медианное значение каждой группы
    nan_days_employed = df[df['days_employed'].isna() == True]
    for i in nan_days_employed['income_type'].unique():
        df.loc[df['income_type'] == i,'days_employed'] = df['days_employed'].fillna(median_days_employed[i])
        df.loc[df['income_type'] == i, 'total_income'] = df['total_income'].fillna(median_total_income[i])
        
    return df

In [18]:
data = filling(data, median_days, median_income) #применение функции к нашим данным

In [19]:
data.isna().sum() #проверка на количество пропусков в столбцах

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод:** 
Пропуски были найдены, с помощью метода isna(), только в двух столбцах - это days_employed и total_income, и эти пропуски были взаимосвязаны. Это может быть связано с возможность пользователям не сообщать свой трудовой стаж и ежемесячный доход или эти два значения могут быть связаны какой-то формулой, поэтому возникает ситуация, что если о человеке не попадает информация в один столбец, то и второй столбец остается незаполненным.

Пропуски были заполнены с помощью функции, которая высчитывает медиану каждого типа занятости по количеству дней трудового стажа и по ежемесечному доходу, а потом в зависимости от типа занятости человека вставляет ему нужное медианное значение вместо пропуска.

### Замена типа данных

In [20]:
data['days_employed'] = data['days_employed'].astype('int') #заменяем в столбце days_employed тип данных float на тип данных int

data.info() #проверяем изменился ли тип данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


In [21]:
data.info() #проверяем изменился ли тип данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


**Вывод:**
Так как нам нужно перевести данные в толбце days_employed в целочисленный тип данных, то для перевода я использовала метод astype().

### Обработка дубликатов

In [22]:
data['education'] = data['education'].str.lower() #приводим данные в столбце education к нижнему регистру
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [23]:
data.duplicated().sum() #информация о количестве явных дубликатов

71

In [24]:
data = data.drop_duplicates().reset_index(drop = True) #удаляем явные дубликаты
data.duplicated().sum() #проверяем количество дубликатов повторно

0

In [25]:
data.duplicated().sum() #проверяем количество дубликатов повторно

0

**Вывод:**

Для поиска явных дубликатов был использова метод duplicated(), этим методом было обнаружено 71 дубликат и с помощью метода drop_duplcates() удалили дубликаты. 

Появление дубликатов может быть связано с тем, что информация в файл могла собираться из разных источников и даные о некоторых клиентов находились в нескольких источниках. Или дубликаты могут быть связаны с ошибками пользователей при занесении информации в файл.

### Лемматизация

In [26]:
from pymystem3 import Mystem #импортируем модуль pymystem3
m = Mystem()

unique_purpose = data['purpose'].unique() #записываем уникальные значения столбца purpose в переменную unique_purpose

list_df = [] #создаем пустой список

for purpose in unique_purpose:     #цикл, в котором лемматизируются все цели получения кредита
    lemmas = m.lemmatize(purpose)
    list_df.append([purpose, lemmas]) #цель получения кредита и лемматизацию цели добавляем в список list_df
            
columns_df = ['purpose','lemmas_purpose'] #записывем наименование колонок будущего датафрейма

df_with_lemmas = pd.DataFrame(data = list_df, columns = columns_df) #создание датафрейма

<div class="alert alert-success">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>

А можно было вот так 👇 Без мержа)

```python
data['lemmas_purpose'] = data['purpose'].apply( m.lemmatize)
```

In [27]:
df_with_lemmas #в столбце lemmas_purpose находится результат лемматизации каждой цели

,purpose,lemmas_purpose
0,покупка жилья,"[покупка, , жилье, \n]"
1,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,дополнительное образование,"[дополнительный, , образование, \n]"
3,сыграть свадьбу,"[сыграть, , свадьба, \n]"
4,операции с жильем,"[операция, , с, , жилье, \n]"
5,образование,"[образование, \n]"
6,на проведение свадьбы,"[на, , проведение, , свадьба, \n]"
7,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]"
8,покупка недвижимости,"[покупка, , недвижимость, \n]"
9,покупка коммерческой недвижимости,"[покупка, , коммерческий, , недвижимость, \n]"


In [28]:
data = data.merge(df_with_lemmas) # объединяем таблицы по столбцу purpose
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas_purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,"[покупка, , жилье, \n]"
1,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,"[покупка, , жилье, \n]"
2,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,"[покупка, , жилье, \n]"
3,0,1548,48,среднее,1,женат / замужем,0,F,компаньон,0,157245.786233,покупка жилья,"[покупка, , жилье, \n]"
4,0,413,41,среднее,1,женат / замужем,0,M,госслужащий,0,118551.607829,покупка жилья,"[покупка, , жилье, \n]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,379878,52,среднее,1,женат / замужем,0,F,пенсионер,0,92997.470761,заняться высшим образованием,"[заниматься, , высокий, , образование, \n]"
21450,0,365213,67,высшее,0,женат / замужем,0,F,пенсионер,0,118514.486412,заняться высшим образованием,"[заниматься, , высокий, , образование, \n]"
21451,3,3039,26,высшее,0,женат / замужем,0,F,компаньон,0,302642.330826,заняться высшим образованием,"[заниматься, , высокий, , образование, \n]"
21452,0,1117,47,среднее,1,гражданский брак,1,F,сотрудник,0,231833.290488,заняться высшим образованием,"[заниматься, , высокий, , образование, \n]"


**Вывод:**

Используя метод lemmatize() библиотеки pymystem3 все уникальные значения столбца purpose были лемматизированы. Так же была создана таблица, где каждой цели соответствует её лемматизированное значение.

Посмотрев на датафрейм с лемматизированными целями, можно выделить часто встречаемые слова в каждой цели это: жилье, недвижимость,  автомобиль, образование, свадьба. В следующем блоке заменим столбец с лемматизированной целью на одно из этих слов.

### Категоризация данных

In [29]:
def categories_purpose(row):  #функция для категоризации столбца lemmas_purpose
    categories = ['жилье', 'недвижимость', 'автомобиль', 'образование', 'свадьба']
    
    for word in categories:
        if word in row:
            return word   

In [30]:
data['lemmas_purpose'] = data['lemmas_purpose'].apply(categories_purpose) #применяю функцию categories_purpose
data['lemmas_purpose'].unique()

array(['жилье', 'автомобиль', 'образование', 'свадьба', 'недвижимость'],
      dtype=object)

In [32]:
max_income = int(data['total_income'].max()) #максимальное знаяение ежемесячного дохода
min_income = int(data['total_income'].min()) #минимальное значение ежемесячного дохода
min_interval = data['total_income'].median() #если ежемесячный доход ниже значения этой переменной - будем считать его низким
max_interval =  data['total_income'].median() + (max_income - min_income)/100*20 #если доход выше значения этой переменной -
# будем считать его высоким

In [33]:
def categories_of_income(row): #функция для категоризации
    if  row <= min_interval: #row <= 142000:
        return 'низкая'
    
    if  min_interval <row <= max_interval: #142000 < row <= 300000:
        return 'средняя'
    
    if  max_interval < row: #300000 < row:
        return 'высокая'
    
    
data['category_total_income'] = data['total_income'].apply(categories_of_income) #создаем новый столбец с категориями
#ежемесячного дохода

In [34]:
data['category_total_income'].unique() #проверяем уникальные значения нового столбца

array(['средняя', 'низкая', 'высокая'], dtype=object)

**Вывод:** 
Категоризация данных заключается в объединении избранных данных в произвольные группы по определенному критерию. Для распределения данных по группам я использовала функции categories_purpose и categories_of_income. Категория данных была записана в новый столбец с помощью функции apply().

В функции categories_purpose данные объединяются в зависимоссти от того входит ли в леммтизированную цель одно из слов находящихся в переменной categories.

В функции categories_of_income я объединила данные следующим образом:
- все данные, которые ниже медианного ежмесяного дохода, считаются низким доходом;
- все данные, которые выше ежемесячного медианного дохода + 20% от размаха, считаются высоким доходом;
- все данные, которые больше медианного ежмесяного дохода и меньше медианного ежемесячного дохода + 20% от размаха, считаются средним доходом.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [35]:
pvt_children = data.pivot_table(index = 'children', values ='debt', aggfunc = ['count', 'sum', 'mean']) #строим сводную таблицу,
#сгруппированную по столбцу children, которая отображает количество клиентов и общую сумму клиентов с задолженностью 
#по каждой группе
pvt_children

,count,sum,mean
,debt,debt,debt
children,,,
0,14091,1063,0.075438
1,4855,445,0.091658
2,2052,194,0.094542
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000
20,76,8,0.105263


In [36]:
data.pivot_table(index = 'children', 
                 values ='debt', 
                 aggfunc = ['count', 'sum', 'mean']).style.format({('mean', 'debt'): '{:.2%}'})

,count,sum,mean
,debt,debt,debt
children,,,
0,14091,1063,7.54%
1,4855,445,9.17%
2,2052,194,9.45%
3,330,27,8.18%
4,41,4,9.76%
5,9,0,0.00%
20,76,8,10.53%


**Вывод:**
Группа клиентов без детей имеет более низкий процет должников по сравнению с остальными группами - это 7%. В то время как у групп клиентов с детьми процент должников составляет от 8 до 10 процентов. Исключение составляет группа клиентов с пятью детьми, у них 0% должников, это может быть связано с тем, что у банка маленькое количество таких клиентов.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [37]:
pvt_family_status = data.pivot_table(index = 'family_status', 
                                     values ='debt', 
                                     aggfunc = ['count', 'sum', 'mean']).style.format({('mean', 'debt'): '{:.2%}'}) 
#строим сводную  таблицу, сгруппированную по столбцу family_status, которая отображает количество клиентов и общую 
#сумму клиентов с задолженностью по каждой группе
pvt_family_status

,count,sum,mean
,debt,debt,debt
family_status,,,
Не женат / не замужем,2810,274,9.75%
в разводе,1195,85,7.11%
вдовец / вдова,959,63,6.57%
гражданский брак,4151,388,9.35%
женат / замужем,12339,931,7.55%


In [38]:
data.pivot_table(index = 'family_status', 
                 values ='debt', 
                 aggfunc = ['count', 'sum', 'mean']).style.format({('mean', 'debt'): '{:.2%}'})

,count,sum,mean
,debt,debt,debt
family_status,,,
Не женат / не замужем,2810,274,9.75%
в разводе,1195,85,7.11%
вдовец / вдова,959,63,6.57%
гражданский брак,4151,388,9.35%
женат / замужем,12339,931,7.55%


**Вывод:** Самый низкий процент должников у групп с семейным положением: в разводе, вдовец/вдова, женат/замужем - от 6 до 7 процентов. Самый высокий процент должников у групп с семейным положением не женат/не замужем и гражданский брак около 9%. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [39]:
pvt_category_total_income = data.pivot_table(index = 'category_total_income', 
                                             values ='debt', 
                                             aggfunc = ['count', 'sum', 'mean']).style.format({('mean', 'debt'): '{:.2%}'})  
#строим сводную таблицу, сгруппированную по столбцу category_total_income, которая отображает количество клиентов и общую сумму 
#клиентов с задолженностью по каждой группе
pvt_category_total_income

,count,sum,mean
,debt,debt,debt
category_total_income,,,
высокая,117,8,6.84%
низкая,10843,910,8.39%
средняя,10494,823,7.84%


**Вывод:** Клиенты с низким уровнем дохода чаще всего оказываются должниками. Реже всего должниками оказываются клиенты с высоким уровнем дохода.

- Как разные цели кредита влияют на его возврат в срок?

In [40]:
pvt_lemmas_purpose = data.pivot_table(index = 'lemmas_purpose', 
                                      values ='debt', 
                                      aggfunc = ['count', 'sum', 'mean']).style.format({('mean', 'debt'): '{:.2%}'}) #строим сводную 
#таблицу, сгруппированную по столбцу lemmas_purpose, которая отображает количество клиентов и общую сумму клиентов с 
#задолженностью по каждой группе
pvt_lemmas_purpose

,count,sum,mean
,debt,debt,debt
lemmas_purpose,,,
автомобиль,4306,403,9.36%
жилье,4460,308,6.91%
недвижимость,6351,474,7.46%
образование,4013,370,9.22%
свадьба,2324,186,8.00%


**Вывод** Клиенты, которые беру кредит на автомобиль и образование, чаще всего оказываются должниками по сравнению с остальными группами. Клиенты, которые берут кредит на жилье реже всего оказываются должниками.

## Шаг 4. Общий вывод

Перед тем как приступить к исследованию, необходимо было выполнить предобработку данных:
1. Данные в одном из столбцов были приведены к нижнему регистру.
2. Данные в столбце days_employed были заменены только на положительные числа, а затем приведены к целочисленному типу данных.
3. Были заполнены пропуски в данных.
4. Были удалены явные дубликаты.
5. Была проведена категоризация данных.


Целью проекта было узнать влияет ли наличие детей и семейное положение на погашение кредита в срок:
1. Наличие детей влияет на погашение кредита в срок. Процент должников среди клиентов, у которых есть дети, выше чем у клиентов без детей.
2. Семейное положение так же влияет на погашение кредита в срок. Клиенты с семейным положением "не женат/не замужем" и "гражданский брак" чаще оказываются должниками, чем группы клиентов с другим семейным положением.
3. Таже была исследована зависимость между уровнем дохода и возвращением кредита в срок. Было выявлено, что клиенты с низким уровнем дохода чаще оказываются должниками. А клиенты с высоким уровнем дохода - реже.
4. При исследовании влияния цели на возврат кредита в срок было выявлено, что клиенты берущие кредит на автомобиль и образование чаще оказываются должниками. А клиенты берущие кредит на жтлье реже всего оказываются должниками.